In [9]:
import requests
import pandas as pd
import urllib.parse
import json
from datetime import datetime
from pprint import pprint

class ssc:

    def __init__(self,id=''):

        self.id = id

    #Be able to get data for a specific table []
    #Be able to get data for all tables in a specific category []
    #Be able to get data for all tables []
    #Be able to browse or search tables by ID, name, category

    def get_metadata(id, timeout=30, lang='all'):
        """
        Retrieves metadata for a cube (table) from Statistics Canada WDS.

        Args:
            id (int or str): The product ID (PID) of the cube (e.g., 35100003).
            timeout (float): Timeout in seconds for the HTTP request.
            lang (str): Language preference for the metadata ('en', 'fr', or 'all').

        Returns:
            dict: JSON response object converted to Python dict.
        """

        endpoint = "https://www150.statcan.gc.ca/t1/wds/rest/getCubeMetadata"

        # Prepare payload as list of dicts, per documentation example. :contentReference[oaicite:2]{index=2}
        payload = [
            {
                "productId": str(id)
            }
        ]

        headers = {
            "Content-Type": "application/json"
        }

        def _remove_lang(obj,language):
            
            if isinstance(obj, dict):
                
                obj = {k: _remove_lang(v,language) for k, v in obj.items() if not k.endswith(language)}

                for k in obj.keys():
                    if isinstance(obj[k],list):
                        if len(obj[k]) > 0:
                            if isinstance(obj[k][0],dict):
                                print(k)
                                obj[k] = [_remove_lang(i,language) for i in obj[k]]
                return obj
            
            else:
                return obj

        try:
            response = requests.post(endpoint, headers=headers, json=payload, timeout=timeout)
            response.raise_for_status()
            data = response.json()

            # Check for status
            if data[0]["status"] != "SUCCESS":
                raise RuntimeError(f"Request returned non-SUCCESS status: {data}")
            
            if lang == 'en':
                return _remove_lang(data[0]['object'],'Fr')
            
            elif lang == 'fr':
                return _remove_lang(data[0]['object'],'En')
            
            else:
                return data[0]['object']
        
        except requests.RequestException as e:
            raise RuntimeError(f"HTTP request failed: {e}") from e
        
        except json.JSONDecodeError as e:
            raise RuntimeError(f"Failed to parse JSON response: {e}") from e

    def get_table(id='',periods=1,full=True,enc='%2C1%2C2',region_type=''):

        url = f'https://www150.statcan.gc.ca/t1/tbl1/en/dtl!downloadDbLoadingData-nonTraduit.action?pid={id}01&latestN={periods}&startDate=&endDate=&csvLocale=en&selectedMembers={enc}&checkedLevels=0D1%2C0D1%2C0D2'
        
        if full: print('Warning: Unfiltered tables can be very large')
        print(url)
        
        df = pd.read_csv(url)
        if region_type != '':
            df = df[df.DGUID.str[6:9] == region_type]

        display(df)

    def _parse_dim(x):

        tmp = {}
        sel = []

        for i in x:

            tmp[i['dimensionNameEn']] = i['hasUOM']
            sel.append([1])
        
        return tmp,sel

    def list_tables():

        url = "https://www150.statcan.gc.ca/t1/wds/rest/getAllCubesList"
        resp = requests.get(url)
        tables = resp.json()

        df = pd.DataFrame(tables)
        df = df[[x for x in df.columns if x if 'Fr' not in x]]
        df = df[df['archived'] == '2']

        archived_tables = df[df['archived'] == '1']
        active_tables = df[df['archived'] == '2']

        card_css = "border:1px solid black;margin:10px;padding:10px;background:white;max-width:200px;border-radius:3px;color:black;"
        n = 2

        count = 0

        for p in tables:

            id = p['productId']
            endDate = datetime.fromisoformat(p['cubeEndDate'].replace("Z", "+00:00")).replace(tzinfo=None)

            if p['archived'] == '2':

                metadata = get_metadata(id)
                num_dimensions = len(metadata['dimension'])
                data = metadata['dimension']

                tablename = p['cubeTitleEn']
                dim = p['dimensions']
                dim,selected = _parse_dim(dim)

                raw = json.dumps(selected, separators=(',',':'))
                enc = urllib.parse.quote(raw)

                num_param = len(data)

                attributes = {}
                attributes['Table Name'] = tablename

                for i in range(0,num_param):

                    sub_size = len(data[i]['member'])
                    top_name = data[i]['dimensionNameEn']

                    for j in range(0,sub_size):
                        
                        #print(data[i]['member'][j])

                        name = data[i]['member'][j]['memberNameEn']
                        classification = data[i]['member'][j]['classificationCode']

                        if top_name in attributes:
                            attributes[top_name].append(name)
                        else:
                            attributes[top_name] = [name]

                print(id,' - ',tablename,'\n',attributes)
                #get_table(id=id,n=n,enc=enc)#region_type='503')
                
                count += 1

                if count > 5:
                    break
                    

#meta = ssc.get_metadata(id=35100003,lang='en')
#pprint(meta)
df = ssc.get_table(id=35100003)



https://www150.statcan.gc.ca/t1/tbl1/en/dtl!downloadDbLoadingData-nonTraduit.action?pid=3510000301&latestN=1&startDate=&endDate=&csvLocale=en&selectedMembers=%2C1%2C2&checkedLevels=0D1%2C0D1%2C0D2


HTTPError: HTTP Error 503: Service Temporarily Unavailable